The Task: 

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

Lets get the imports

In [39]:
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
import os

load gemini keys

In [40]:

load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')

Lets setup system prompt

In [1]:
system_prompt = """
You are a technical assistant. When asked technical questions, you answer as an experienced engineer, you make complex concept simple by using analogies and examples.
"""


Let's setup audio output

In [75]:
audio_client = OpenAI()
def talker(message):
    response = audio_client.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="onyx",    # Also, try replacing onyx with alloy or coral
      input=message
    )
    return response.content

lets setup the chat function 

In [83]:
def chat(message,history,additional_inputs):
    history = [{"role":h['role'],"content":h["content"]} for h in history]
    messages = [{'role':'system','content':system_prompt}] + history + [{'role':'user','content':message}]
    client = OpenAI() if additional_inputs == "OpenAI" else OpenAI(base_url= "https://generativelanguage.googleapis.com/v1beta/openai/",api_key=google_api_key)
    stream = client.chat.completions.create(
        model= "gpt-4o-mini" if additional_inputs == "OpenAI" else "gemini-2.0-flash",
        messages=messages,
        stream=True
    )
    history = history + [
        {"role": "assistant", "content": ""}
    ]
    response_text = ""
    for chunk in stream:
        response_text += chunk.choices[0].delta.content or ''
        history[-1]["content"] = response_text
        yield history,None

    yield history, talker(response_text)
    
    


Lets pipe this to a customized Gradio view

In [ ]:
available_models = ["OpenAI","Gemini"]

with gr.Blocks() as ui:
    chatbot = gr.Chatbot(type="messages",height=400)
    audioOutput = gr.Audio(autoplay=True)  
    with gr.Row():
        chatInput = gr.Textbox(show_label=False,interactive=True,scale=8,placeholder="Enter your message here...",)
        model_dropdown = gr.Dropdown(choices=available_models,value="OpenAI",interactive=True,scale=2,show_label=False)
    chatInput.submit(fn=chat,inputs=[chatInput,chatbot,model_dropdown],outputs=[chatbot,audioOutput])

ui.launch()